# Biopython tutorial and sequence objects

In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

### Using biopython to create sequence motifs

In [5]:
# AGAR workshop script to analyze motifs
from Bio import motifs
from Bio.Seq import Seq

# suppose we have sequences for motifs of a given transcription factor binding site
instances = [Seq("TACAA"), Seq("TACGC"), Seq("TACAC"), Seq("TACCC"), Seq("AACCC"), Seq("AATGC"), Seq("AATGC")]

# using the motifs module create m motifs of the instances of the sequences
m = motifs.create(instances)

# print all motifs
print(m)

# nucleotide counts can be accessed on whole
print(m.counts)

# or by nucleotide, by using the base name as the dict key
print(m.counts['A'])

# or access all elements in the first two positions of the motifs
print(m.counts[:,0:1])

TACAA
TACGC
TACAC
TACCC
AACCC
AATGC
AATGC

        0      1      2      3      4
A:   3.00   7.00   0.00   2.00   1.00
C:   0.00   0.00   5.00   2.00   6.00
G:   0.00   0.00   0.00   3.00   0.00
T:   4.00   0.00   2.00   0.00   0.00

[3, 7, 0, 2, 1]
        0
A:   3.00
C:   0.00
G:   0.00
T:   4.00



### To access Entrez databases:

In [6]:
from Bio import Entrez
Entrez.email = "your@email.com"
handle = Entrez.einfo()
record =  Entrez.read(handle)
record.keys()
for l in record['DbList']:
    print(l)

pubmed
protein
nuccore
ipg
nucleotide
nucgss
nucest
structure
sparcle
genome
annotinfo
assembly
bioproject
biosample
blastdbinfo
books
cdd
clinvar
clone
gap
gapplus
grasp
dbvar
gene
gds
geoprofiles
homologene
medgen
mesh
ncbisearch
nlmcatalog
omim
orgtrack
pmc
popset
probe
proteinclusters
pcassay
biosystems
pccompound
pcsubstance
pubmedhealth
seqannot
snp
sra
taxonomy
biocollections
unigene
gencoll
gtr


### Fetching gene information using a Gene ID

In [8]:
from Bio import Entrez
import time
Entrez.email ="eigtw59tyjrt403@gmail.com"

# epas1 ID
epas1_id = 'U81984.1'

# upload a list of IDs beforehand 
gis=[epas1_id]

# form a request and choose the database
request = Entrez.epost("nucleotide",id=",".join(map(str,gis)))

# fetch the results
result = Entrez.read(request)
webEnv = result["WebEnv"]
queryKey = result["QueryKey"]

# request the results in xml format
handle = Entrez.efetch(db="nucleotide",retmode="xml", webenv=webEnv, query_key=queryKey)

# parse the results using biopython
results = []
for r in Entrez.parse(handle):
    # Grab the GI 
    try:
        gi=int([x for x in r['GBSeq_other-seqids'] if "gi" in x][0].split("|")[1])
    except ValueError:
        gi=None
    print(">GI ",gi," "+r["GBSeq_primary-accession"]+" "+r["GBSeq_definition"]+"\n")
    results.append(r)
for key in r.keys():
    print(key)


>GI  1805267  U81984 Human endothelial PAS domain protein 1 (EPAS1) mRNA, complete cds

GBSeq_locus
GBSeq_length
GBSeq_strandedness
GBSeq_moltype
GBSeq_topology
GBSeq_division
GBSeq_update-date
GBSeq_create-date
GBSeq_definition
GBSeq_primary-accession
GBSeq_accession-version
GBSeq_other-seqids
GBSeq_source
GBSeq_organism
GBSeq_taxonomy
GBSeq_references
GBSeq_feature-table
GBSeq_sequence


In [8]:
epas1_dna = r['GBSeq_sequence']
print('EPAS1 genomic DNA:', epas1_dna)

cctgactgcgcggggcgctcgggacctgcgcgcacctcggaccttcaccacccgcccgggccgcggggagcggacgagggccacagccccccacccgccagggagcccaggtgctcggcgtctgaacgtctcaaagggccacagcgacaatgacagctgacaaggagaagaaaaggagtagctcggagaggaggaaggagaagtcccgggatgctgcgcggtgccggcggagcaaggagacggaggtgttctatgagctggcccatgagctgcctctgccccacagtgtgagctcccatctggacaaggcctccatcatgcgactggaaatcagcttcctgcgaacacacaagctcctctcctcagtttgctctgaaaacgagtccgaagccgaagctgaccagcagatggacaacttgtacctgaaagccttggagggtttcattgccgtggtgacccaagatggcgacatgatctttctgtcagaaaacatcagcaagttcatgggacttacacaggtggagctaacaggacatagtatctttgacttcactcatccctgcgaccatgaggagattcgtgagaacctgagtctcaaaaatggctctggttttgggaaaaaaagcaaagacatgtccacagagcgggacttcttcatgaggatgaagtgcacggtcaccaacagaggccgtactgtcaacctcaagtcagccacctggaaggtcttgcactgcacgggccaggtgaaagtctacaacaactgccctcctcacaatagtctgtgtggctacaaggagcccctgctgtcctgcctcatcatcatgtgtgaaccaatccagcacccatcccacatggacatccccctggatagcaagaccttcctgagccgccacagcatggacatgaagttcacctactgtgatgacagaatcacagaactgattggttaccaccctgaggagctgcttggccgctcagcctatgaattctaccatgcgct

### Do not perform these operations during peak NCBI hours, BLAST can be run locally too

In [9]:
from Bio.Blast import NCBIWWW

result_handle = NCBIWWW.qblast("blastn", "nt", str(epas1_id))

with open(str(epas1_id) + "_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

result_handle.close()

In [53]:
result_handle = open(str(epas1_id) + "_blast.xml")

from Bio.Blast import NCBIXML
blast_records = NCBIXML.read(result_handle)
E_VALUE_THRESH = 0.04

for alignment in blast_records.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query[0:75] + '...')
            print(hsp.match[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')
            
from Bio import SearchIO     
blast_qresult = SearchIO.read(str(epas1_id) + "_blast.xml", 'blast-xml')

# print(blast_qresult)
blast_hit = blast_qresult[3]
print(blast_hit)

****Alignment****
sequence: gi|1805267|gb|U81984.1|HSU81984 Human endothelial PAS domain protein 1 (EPAS1) mRNA, complete cds
length: 2818
e value: 0.0
CCTGACTGCGCGGGGCGCTCGGGACCTGCGCGCACCTCGGACCTTCACCACCCGCCCGGGCCGCGGGGAGCGGAC...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
CCTGACTGCGCGGGGCGCTCGGGACCTGCGCGCACCTCGGACCTTCACCACCCGCCCGGGCCGCGGGGAGCGGAC...
****Alignment****
sequence: gi|262527236|ref|NM_001430.4| Homo sapiens endothelial PAS domain protein 1 (EPAS1), mRNA
length: 5184
e value: 0.0
CCTGACTGCGCGGGGCGCTCGGGACCTGCGCGCACCTCGGACCTTCACCACCCGCCCGGGCCGCGGGGAGCGGAC...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
CCTGACTGCGCGGGGCGCTCGGGACCTGCGCGCACCTCGGACCTTCACCACCCGCCCGGGCCGCGGGGAGCGGAC...
****Alignment****
sequence: gi|30410994|gb|BC051338.1| Homo sapiens endothelial PAS domain protein 1, mRNA (cDNA clone MGC:59860 IMAGE:6305604), complete cds
length: 5011
e value: 0.0
CCTGACTGCGCGGGGCGCTCGGGACCTGCGCGCACCTCGGACCTTC

In [54]:
from Bio import motifs
from Bio.Seq import Seq
instances = [Seq("TACAA"),
              Seq("TACGC"),
              Seq("TACAC"),
              Seq("TACCC"),
              Seq("AACCC"),
              Seq("AATGC"),
              Seq("AATGC"),
             ]

m = motifs.create(instances)
print(m)
print(m.counts)

print(m.consensus)

print(m.anticonsensus)

m.weblogo("mymotif.png",kwargs=['stack_width:medium','stacks_per_line:40','alphabet:alphabet_dna','ignore_lower_case:True','unit_name:bits','first_index:1',
'logo_start:1',
'composition:comp_auto',
'show_xaxis:False',
'show_yaxis:False',
'yaxis_scale:auto',
'yaxis_tic_interval:1.0',
'color_scheme:color_auto',])




TACAA
TACGC
TACAC
TACCC
AACCC
AATGC
AATGC

        0      1      2      3      4
A:   3.00   7.00   0.00   2.00   1.00
C:   0.00   0.00   5.00   2.00   6.00
G:   0.00   0.00   0.00   3.00   0.00
T:   4.00   0.00   2.00   0.00   0.00

TACGC
GGGTG


### Expasy Prosite

In [55]:
from Bio import ExPASy
from Bio import SwissProt
accessions = ["Q99814"]
for accession in accessions:
    handle = ExPASy.get_sprot_raw(accession)
    record = SwissProt.read(handle)
    handle = ExPASy.get_sprot_raw(accession)
    text= handle.read()
# print(text)    
data = [x.strip() for x in text.split('\n')]
aa = data[-17:]
AA = ''
for a in aa:
    AA = str(AA) + str(a)
AA = AA.split(' ')
new_aa = ''.join(AA)[0:-2]
print('Epas1 amino acid sequence:\n'+str(new_aa))

Epas1 amino acid sequence:
MTADKEKKRSSSERRKEKSRDAARCRRSKETEVFYELAHELPLPHSVSSHLDKASIMRLAISFLRTHKLLSSVCSENESEAEADQQMDNLYLKALEGFIAVVTQDGDMIFLSENISKFMGLTQVELTGHSIFDFTHPCDHEEIRENLSLKNGSGFGKKSKDMSTERDFFMRMKCTVTNRGRTVNLKSATWKVLHCTGQVKVYNNCPPHNSLCGYKEPLLSCLIIMCEPIQHPSHMDIPLDSKTFLSRHSMDMKFTYCDDRITELIGYHPEELLGRSAYEFYHALDSENMTKSHQNLCTKGQVVSGQYRMLAKHGGYVWLETQGTVIYNPRNLQPQCIMCVNYVLSEIEKNDVVFSMDQTESLFKPHLMAMNSIFDSSGKGAVSEKSNFLFTKLKEEPEELAQLAPTPGDAIISLDFGNQNFEESSAYGKAILPPSQPWATELRSHSTQSEAGSLPAFTVPQAAAPGSTTPSATSSSSSCSTPNSPEDYYTSLDNDLKIEVIEKLFAMDTEAKDQCSTQTDFNELDLETLAPYIPMDGEDFQLSPICPEERLLAENPQSTPQHCFSAMTNIFQPLAPVAPHSPFLLDKFQQQLESKKTEPEHRPMSSIFFDAGSKASLPPCCGQASTPLSSMGGRSNTQWPPDPPLHFGPTKWAVGDQRTEFLGAAPLGPPVSPPHVSTFKTRSAKGFGARGPDVLSPAMVALSNKLKLKRQLEYEEQAFQDLSGGDPPGGSTSHLMWKRMKNLRGGSCPLMPDKPLSANVPNDKFTQNPMRGLGHPLRHLPLPQPPSAISPGENSKSRFPPQCYATQYQDYSLSSAHKVSGMASRLLGPSFESYLLPELTRYDCEVNVPVLGSSTLLQGGDLLRALDQAT
